In [1]:
import asyncio
import numpy as np
import quaternion

from intrepid_environment.simulator import Simulator

In [2]:
import matplotlib.pyplot as plt

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
sim = Simulator()
await sim.connect()

0

In [5]:
async def restart_drone():
    await sim.rpc("session.restart")
    entity = await sim.rpc("map.spawn_uav", {
        "robot_id": 0,
        "position": {
            "x": 0,
            "y": 0,
            "z": 0,
        },
    })
    await sim.rpc("session.run")
    return entity

In [6]:
entity = await restart_drone()

Setup trajectory for drone to follow.

In [7]:
from quadcopter_control.trajectory import Trajectory, TrajectoryPoint

In [8]:
async def sim_step(motors):
    (sim_time, _, drone_state) = await asyncio.gather(
            sim.step(),
            sim.rpc(f"object_{entity}.actuator_control", motors),
            sim.rpc("script.eval", {
                "code": """{
                    position = sim.object.position(ARGS),
                    attitude = sim.object.rotation_quat(ARGS),
                    velocity = sim.object.linear_velocity(ARGS),
                    omega = sim.object.angular_velocity(ARGS),
                    acceleration = sim.object.acceleration(ARGS),
                }""",
                "args": entity,
            }),
        )

    acceleration = np.array([drone_state['acceleration']['y'],
                             drone_state['acceleration']['x'],
                             drone_state['acceleration']['z']], dtype=np.float64)

    position = np.array([drone_state['position']['y'],
                        drone_state['position']['x'],
                        drone_state['position']['z']], dtype=np.float64)

    velocity = np.array([drone_state['velocity']['y'],
                        drone_state['velocity']['x'],
                        drone_state['velocity']['z']], dtype=np.float64)

    attitude = np.quaternion(drone_state['attitude'][3],
                             drone_state['attitude'][0],
                             drone_state['attitude'][1],
                             drone_state['attitude'][2])

    omega = np.array([drone_state['omega']['xy'],
                     drone_state['omega']['zx'],
                     drone_state['omega']['yz']], dtype=np.float64)

    return {
        'time': sim_time,
        'acceleration': acceleration,
        'position': position,
        'velocity': velocity,
        'attitude': attitude,
        'omega': omega
    }

In [9]:
async def get_trajectory_with_initial_position():
    state = await sim_step([0, 0, 0, 0])

    starting_point = TrajectoryPoint(
        time=state['time'],
        acceleration=state['acceleration'],
        attitude=state['attitude'],
        omega=state['omega'],
        position=state['position'],
        velocity=state['velocity'],
    )

    trajectory = Trajectory()
    trajectory.add_point(starting_point)

    return trajectory

In [10]:
trajectory_time_offset = 0
dp = np.zeros(3)

def get_next_trajectory_point(trajectory, sim_time, dt, dp):
    pt = trajectory.next_trajectory_point((sim_time + dt) / 1000.0)
    pt.position += dp
    return pt

In [11]:
from quadcopter_control.controller import QuadcopterController, calculate_drone_moment_of_inertia

In [12]:
DRONE_HEIGHT = 0.028
DRONE_RADIUS = 0.05
DRONE_MASS = 0.041

In [13]:
(Ixx, Iyy, Izz) = calculate_drone_moment_of_inertia(DRONE_HEIGHT, DRONE_RADIUS, DRONE_MASS)
(Ixx, Iyy, Izz)

(2.830366666666667e-05, 2.830366666666667e-05, 5.125000000000001e-05)

## Scenario -1: Determine Drone Motor / to Newton Rate

In [14]:
# drone_force = 9.81 * DRONE_MASS; drone_force

In [15]:
# positions = []
# forces = []

In [16]:
# for force in np.arange(50, 51, 0.01):
#     print("trying force", force)
#     for _ in range(50):
#         state = await sim_step([force, force, force, force])
#         forces.append(force)
#         positions.append(state['position'][2])

In [17]:
# plt.plot(positions[:1250])
# plt.plot(forces[:1250])
# plt.title("z", fontsize=20)
# plt.show()

In [18]:
scale = 0.40221 / (50.2*4); scale

0.0020030378486055778

500 = 1 newton

## Scenario 0: Zero Gains

Test whether controller runs without controller gains set.

In [19]:
trajectory = await get_trajectory_with_initial_position()

In [20]:
DT_MS = 15.625 # 64 hz

In [21]:
quadcopter_controller = QuadcopterController(
    DT_MS / 1e3,
    DRONE_MASS,
    Ixx, Iyy, Izz,
    l=0.05,
    max_tilt_angle=0.22,
    max_ascent_rate=20.0
)

In [22]:
trajectory = await get_trajectory_with_initial_position()


async def control_step(state):
    motors = quadcopter_controller.run_control(
        traj_pt=trajectory.next_trajectory_point((state['time'] + DT_MS*1e3)/1e6),
        est_pos=state['position'],
        est_vel=state['velocity'],
        est_omega=state['omega'],
        est_att=state['attitude'],
    )

    return await sim_step(motors.tolist())

## Scenario 1: Altitude Control

Tune gains until drone stays in place. on a desired height.

In [23]:
state = await sim_step([0, 0, 0, 0])
trajectory = await get_trajectory_with_initial_position()

In [24]:
sim_time = (await sim_step([0, 0, 0, 0]))['time']
n_episodes = 200

for i in range(0, n_episodes):
    trajectory.add_point(TrajectoryPoint(
        time=(sim_time + (i*DT_MS*1e3) / 1e6),
        position=np.array([0.0, 0.0, 5.0]),
        velocity=np.zeros(3),
        acceleration=np.zeros(3),
        attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
        omega=np.array([0.1, 0.1, 0.1])
    ))

In [25]:
import time
import matplotlib.pyplot as plt

In [26]:
from tqdm import tqdm

In [ ]:
async def tune_controller(
        kp_pqr=np.array([95.0, 95.0, 6.0]),
        kp_bank=0.0,
        kp_pos_z=0.0,
        kp_vel_z=0.0,
        ki_pos_z=0.0,
        kp_pos_xy=0.0,
        kp_yaw=0.0,
        kp_vel_xy=0.0,
        kappa=1.0
):

    quadcopter_controller.set_gains(
        kp_pqr=kp_pqr,
        kp_bank=kp_bank,
        kp_pos_z=kp_pos_z,
        kp_vel_z=kp_vel_z,
        ki_pos_z=ki_pos_z,
        kp_pos_xy=kp_pos_xy,
        kp_yaw=kp_yaw,
        kp_vel_xy=kp_vel_xy,
        kappa=kappa
    )

    times = []
    positions = []

    state = await sim_step([0, 0, 0, 0])

    for _ in tqdm(range(n_episodes)):
        t = (state['time'] + DT_MS * 1e3) / 1e6
        traj_pt = trajectory.next_trajectory_point(t)

        motors = quadcopter_controller.run_control(
            traj_pt,
            state['position'],
            state['velocity'],
            state['omega'],
            state['attitude'],
        ) / 0.02

        motors = np.clip(motors, 30, 100)
        positions.append(state['position'])
        times.append(times)
        state = await sim_step(motors.tolist())

    print("end")
    return np.array(times), np.array(positions)

In [31]:
entity = await restart_drone()

In [ ]:
(times, positions) = await tune_controller(
    kp_pqr=np.array([0.0, 0.0, 0.0]),
    ki_pos_z=1.5,
    kp_pos_z=0.0,
    kp_vel_z=0.00,
    kp_pos_xy=0.0,
    kp_vel_xy=0.0,
    kp_yaw=0.0,
    kp_bank=0.0
)

plt.plot(times, positions[2])
plt.show()

  2%|▏         | 3/200 [00:00<00:08, 22.27it/s]

pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state


  4%|▍         | 9/200 [00:00<00:08, 22.90it/s]

post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state


  6%|▌         | 12/200 [00:00<00:08, 22.97it/s]

...

 96%|█████████▌| 192/200 [00:08<00:00, 22.80it/s]

post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state


 99%|█████████▉| 198/200 [00:08<00:00, 22.64it/s]

post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state
post sim state
pre sim state


100%|██████████| 200/200 [00:08<00:00, 22.45it/s]